In [18]:
import subprocess
import difflib
import numpy as np
import pandas as pd
import math
import random
from Bio.pairwise2 import format_alignment 
from Bio import pairwise2

In [19]:
master_sequence = 'GAGCCUCCCUGCUCAGCCUUCCCGAAUCCUGCCCUCGGCUUCUUAAUAUAACUGCCUUAAACGUUUAAUUCUACUUGCACCAAAUAGCUAGUUAGAGCAGACCCUCUCUUAAUCCCGUGGGGCUGUGAACGCGGCGGGGCCAGGCCCACGGCACCCUGACUGGCUAAAACUGUUUGUCCCUUUUUAUUUGAAGAUUGAGUUUCCUCGGGGUCUUCUCUGCCCCGACUUGCUCCCCGUGUACCUUGGUCGACUCCGGAGGUUCAGGUGCACGGACACCCUUUCAAGUUCACCCCUACUCCAUCCUCAGACUUUCUUUUCACGGCGAGGCGCACCCCUCCAGCUUCCGUGGGCACUGCGGAUAGACAGGCACACCGCCAAGGAGCCAGAGAGCAUGGCGCAGGGGACUGUGUGGUCCAGGCUUCCUUUGUUUUCUUUCCCCUAAAGAGCUUUGUUUUUCCUAACAGGAUCAGACAGUCUUGGAGUGGCUUACACAACGGGGGCUUGUGGUAUGUGAGCACAGGCUGGGCAGCUGUGAGAGUCCAGAGUGGGGUGGCCCUGGGGACGCUUCCAGGCCAGCGGUUCCCUGCACCCCACCAGCUGAUUUCGAGCGUGGCAGAGGGAAGGAAAGGGGCGAGCGGGCUGGGCAAUGGACCCGACAGGAAACGGGGACUUAGGGGAACACGCUGGAGAUGCCAUGUGUGGCUGCCGAAGGUCACCAUCUCUCCUCAGUGGCUCCCCAGAGCAGGUGCUUUUAAGAACCCUGUUUCCUCUCAGAGCCCAGGGAGAGUCCAAGGACAUGGCGCAUCAGGAAGUGGGACUGCAGGAGUUCUCUGGUGGCCUCGUGCUGUCCCUCUGGCCACUUCUCACUUUAGGGUGGUCAGCGGCAGCUCGCCAUGGCAGUGCCCAUUGGUGCACACUAACCUCAGUGGAAAAGUAACCAUUCCCUGCCUCUUAGAAAGAACUCAUUCUUAGUUUUAGGAGGGUUCCUGUCGCUGAAUCAAGUCGCUGCCCUGGAUGCAGGGCUGGCCUGGGCGACCCUCCAGGGAUGAGGAGCUCAGAAUUCCAGUCUUCUAAUGUCCACGGACACCUCCCCAUCCCUCUAACGUACUGACUAUGUCUUUUGAUUUAGCAUGUCUUCUAUAGACCUUCCAAAGAGACCCACACUGGCACUGUCACCCCCUAGGAGGGAAGGUGAUGGUUGAUGUAGCCCGACGCGCAUCUUGUUAAUCCGUUCUAAUUCCGAGGAGAGUGUGGGUUUAAGAUAACACCUAUUAAUGCAUUGCCACAAUAAUGUGGGGGUAAGAGAAACGCAGGGACGAAACUUCCAGAAACAAACCCUCCAGAUCGUUCCACAGGAGUGUUCGCCCUCCGGUGUGACUGAACGACCGACCUUGCCCAUGGCUUCAUCCAGACAGCACAGCUGCAGUAUGGCUGGACAGAAGCACCUACUGUUCUUGGAUAUUGAAAUAAAAUAAUAAACUUGCAAUGAUCUUU'
master_structure = '...................................................(((((..............(((((.((........)).)).))).((((.((((.............))))))))......)))))....................(((.(((.......))).))).((((......))))...((((....(((((((.......)))))))...)))).((((((((((.(((((....)))....))))))))))))................................................(((((((......))))..))).......................(((.....)))........................................................................((((((.((((...))))..)))))).......(((.....))).......(((((((.........))))))).......................................................................................................................................(((((((((((.....(((((.(((...(((((....(((.((((((......))))))))).)))))...)))..))))).((((....)))).......))))))))))).................................((((((.....((((((.....)))))))))))).......((((((...(((((((((.((......))))))))))).))))))...................................(((.((((.........))))))).........................................................(((((((((....)))))).))).................................................................................((.(((.(((....))).))))).((((.....)))).........(((((((((((...(((((((((((((...))))..))))))))).(((((.((.....))))))).......(((.............))).)))))))))))........................((((......))))(((((((........((((....................)))).....((((((((((((((........))))..))))...)))))).....)))))))...(((((.(((((.((((......))....)).)))))..)))))...................................................'

In [20]:
# indices = [1003,1030]

In [21]:
indices = [650,775]

In [22]:
sequence = master_sequence[indices[0]:indices[1]]
structure = master_structure[indices[0]:indices[1]]

In [23]:
print(sequence)
print(structure)

ACCCGACAGGAAACGGGGACUUAGGGGAACACGCUGGAGAUGCCAUGUGUGGCUGCCGAAGGUCACCAUCUCUCCUCAGUGGCUCCCCAGAGCAGGUGCUUUUAAGAACCCUGUUUCCUCUCAGA
.......(((((((((((.....(((((.(((...(((((....(((.((((((......))))))))).)))))...)))..))))).((((....)))).......)))))))))))......


In [24]:
def find_pairs(structure):
    stack = []
    pairs = [None]*len(structure)
    for i, c in enumerate(structure):
        if c == '(':
            stack.append(i)
        elif c == ')':
            left = stack.pop()
            right = i
            pairs[right] = left
            pairs[left] = right
        elif c == '.':
            pass
    return pairs

In [25]:
pairs = find_pairs(structure)
print(pairs)

[None, None, None, None, None, None, None, 118, 117, 116, 115, 114, 113, 112, 111, 110, 109, 108, None, None, None, None, None, 87, 86, 85, 84, 83, None, 80, 79, 78, None, None, None, 74, 73, 72, 71, 70, None, None, None, None, 68, 67, 66, None, 65, 64, 63, 62, 61, 60, None, None, None, None, None, None, 53, 52, 51, 50, 49, 48, 46, 45, 44, None, 39, 38, 37, 36, 35, None, None, None, 31, 30, 29, None, None, 27, 26, 25, 24, 23, None, 100, 99, 98, 97, None, None, None, None, 92, 91, 90, 89, None, None, None, None, None, None, None, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, None, None, None, None, None, None]


In [26]:
def find_helices(pairs):
    helices = []
    current_helix = []
    previous = None
    for i in range(len(pairs)):
        if pairs[i] is not None:
            left = i
            right = pairs[i]
            if left < right: # We haven't seen this bp yet
                bp = [left, right]
                if previous is None or len(current_helix) == 0: # our first bp
                    current_helix.append(bp)
                    previous = bp
                elif bp[0] - previous[0] == 1 and bp[1] - previous[1] == -1: # this is a consecutive bp
                    current_helix.append(bp)
                    previous = bp
                else: #non-consecutive bp
                    print(current_helix)
                    helices.append(current_helix)
                    current_helix = [bp]
                    previous = bp
       
        elif len(current_helix) > 1:
            print(current_helix)
            helices.append(current_helix)
            current_helix = []
        else:
            continue
    return helices

In [27]:
helices = find_helices(pairs)
print(helices)

[[7, 118], [8, 117], [9, 116], [10, 115], [11, 114], [12, 113], [13, 112], [14, 111], [15, 110], [16, 109], [17, 108]]
[[23, 87], [24, 86], [25, 85], [26, 84], [27, 83]]
[[29, 80], [30, 79], [31, 78]]
[[35, 74], [36, 73], [37, 72], [38, 71], [39, 70]]
[[44, 68], [45, 67], [46, 66]]
[[48, 65], [49, 64], [50, 63], [51, 62], [52, 61], [53, 60]]
[[89, 100], [90, 99], [91, 98], [92, 97]]
[[[7, 118], [8, 117], [9, 116], [10, 115], [11, 114], [12, 113], [13, 112], [14, 111], [15, 110], [16, 109], [17, 108]], [[23, 87], [24, 86], [25, 85], [26, 84], [27, 83]], [[29, 80], [30, 79], [31, 78]], [[35, 74], [36, 73], [37, 72], [38, 71], [39, 70]], [[44, 68], [45, 67], [46, 66]], [[48, 65], [49, 64], [50, 63], [51, 62], [52, 61], [53, 60]], [[89, 100], [90, 99], [91, 98], [92, 97]]]


In [28]:
def linearfold(test_sequence):
    output = subprocess.check_output('echo "{}" | /usr/local/src/LinearFold/linearfold'.format(test_sequence), shell=True)
    output = output.split('\n')[1]
    structure,energy = output.split(' ')
    energy = float(energy.replace('(', '').replace(')', ''))
    return structure,energy

In [29]:
def compute_bp_recovery(pairs1, pairs2, ignore_helix=None, specific_helix=None):
    """
    Counts how many base pairs are recovered
    assumes pairs1 is the reference
    """
    assert len(pairs1) == len(pairs2)
    assert type(pairs1) == type(pairs2) == list
    recovered = 0
    
    if ignore_helix is not None:
        ignore_helix = [i for sublist in ignore_helix for i in sublist]
        for index in ignore_helix:
            pairs1[index] = None
            pairs2[index] = None
    
    total_helices = float(len([x for x in pairs1 if x is not None]))

    for i in range(len(pairs1)):
        if pairs1[i] == pairs2[i] == None:
            continue
        elif pairs1[i] == pairs2[i]:
            recovered += 1
        elif pairs1[i] is None and pairs2[i] is not None: #False positive
            recovered -= 0.5
    return recovered/total_helices


In [30]:
def compute_bp_disruption(pairs1, pairs2, helix):
    """
    Counts up the number of bps that are disrupted
    assumes pairs1 is the reference
    returns fraction of pairs disrupted
    """
    total_bps = 2*len(helix)
    collapsed = [i for sublist in helix for i in sublist]
    
    disrupted_bps = 0
    for x in collapsed:
        if pairs2[x] is None:
            disrupted_bps += 1
    return disrupted_bps/(float(total_bps))

In [31]:
compute_bp_recovery([None, 1, 3,2,4,None],[None, 1, 3,2,4,None], ignore_helix=[[1,4]])

1.0

In [32]:
compute_bp_disruption([None, 1, 3,2,4,None],[None, 1, None,None,4,None], [[1,4], [2,3]])

0.5

In [33]:
def generate_helix_flips(sequence, structure, helix, gc_rescue=True, scramble_iterations=500):
    print("Generating helix variants for\nsequence: {}\nstructure:{}\nhelix: {}".format(sequence, structure,helix))

    left_indices = [helix[0][0], helix[-1][0]]
    left_original = sequence[left_indices[0]:left_indices[1]+1]
    right_indices = [helix[-1][1], helix[0][1]]
    right_original = sequence[right_indices[0]:right_indices[1]+1]
    
    # Flip the left half of the helix
    left_flipped = list(sequence)
    left_flipped[left_indices[0]:left_indices[1]+1] = left_flipped[left_indices[0]:left_indices[1]+1][::-1]
    left_flipped = ''.join(left_flipped)
    
    alignments = pairwise2.align.globalxx(sequence, left_flipped) 
    print(format_alignment(*alignments[0])) 
    
    # Flip the right half of the helix
    right_flipped = list(sequence)
    right_flipped[right_indices[0]:right_indices[1]+1] = right_flipped[right_indices[0]:right_indices[1]+1][::-1]
    right_flipped = ''.join(right_flipped)
    
    alignments = pairwise2.align.globalxx(sequence, right_flipped) 
    print(format_alignment(*alignments[0]))
    
    # Flip both sides of the helix
    # optionally, GU pairs => CG pairs for extra stability
    rescued = list(sequence)
    rescued[right_indices[0]:right_indices[1]+1] = rescued[right_indices[0]:right_indices[1]+1][::-1]
    rescued[left_indices[0]:left_indices[1]+1] = rescued[left_indices[0]:left_indices[1]+1][::-1]

    for pair in helix:
        left = pair[0]
        right = pair[1]
        if gc_rescue:
            if [rescued[left],rescued[right]] == ['G','U']:
                rescued[left] = 'G'
                rescued[right] = 'C'

            elif [rescued[left],rescued[right]] == ['U','G']:
                rescued[left] = 'C'
                rescued[right] = 'G'
    rescued = ''.join(rescued)
    
    alignments = pairwise2.align.globalxx(sequence, rescued) 
    print(format_alignment(*alignments[0]))
    
    print('left flipped:\n{}\nright flipped:\n{}\nboth flipped\n{}'.format(left_flipped,
                                                                           right_flipped,
                                                                           rescued))
    
    results = {'sequence': sequence,
               'structure': structure,
               'helix': helix,
               'left_WT': left_original,
               'right_WT': right_original,
               'left_flipped': left_flipped,
               'left_flipped_structure': linearfold(left_flipped)[0],
               'right_flipped': right_flipped,
               'right_flipped_structure': linearfold(right_flipped)[0],
               'flipped_rescue': rescued,
               'flipped_rescue_structure': linearfold(rescued)[0]}
    
    disruption_lower_bound = max(compute_bp_disruption(find_pairs(structure), find_pairs(left_flipped), helix),
                                 compute_bp_disruption(find_pairs(structure), find_pairs(right_flipped), helix))

    print("Disruption fraction lower bound: {}".format(disruption_lower_bound))
    # Generate scrambled disrupted helix
    iterations = 0
    scrambled = list(sequence)
    left_sequence = scrambled[left_indices[0]:left_indices[1]+1]
    right_sequence = scrambled[right_indices[0]:right_indices[1]+1]
    scrambled_candidates = []
    iterations = 0
    while iterations < scramble_iterations:
        tmp_scrambled = list(sequence)
        random.shuffle(left_sequence)
        random.shuffle(right_sequence)
        if left_sequence != left_original and right_sequence != right_original:
            tmp_scrambled[left_indices[0]:left_indices[1]+1] = left_sequence
            tmp_scrambled[right_indices[0]:right_indices[1]+1] = right_sequence
            scrambled_candidates.append(''.join(tmp_scrambled))
            iterations += 1

    best_recovery = 0
    best_scrambled_recovery_sequence = None
    for i in scrambled_candidates:
        tmp_structure, tmp_energy = linearfold(i)
        tmp_recovery = compute_bp_recovery(find_pairs(structure), find_pairs(tmp_structure), ignore_helix=helix)
        amount_disrupted = compute_bp_disruption(find_pairs(structure), find_pairs(tmp_structure), helix)
        if  amount_disrupted >= disruption_lower_bound and tmp_recovery > best_recovery:
            best_scrambled_recovery_sequence = i
            best_scrambled_recovery_structure = tmp_structure
            best_recovery = tmp_recovery

    if best_scrambled_recovery_sequence is not None:
        results['scrambled_disruption_sequence'] = best_scrambled_recovery_sequence
        results['scrambled_disruption_structure'] = best_scrambled_recovery_structure
    
    
    print("Generating scrambled rescue")
    # Generate a scrambled rescued helix (both sides are scrambled independently) to search for an optimal structure recovery
    if len(helix) > 5:
        scrambled = list(sequence)
        left_sequence = scrambled[left_indices[0]:left_indices[1]+1]
        right_sequence = scrambled[right_indices[0]:right_indices[1]+1]
        scrambled_candidates = []

        iterations = 0
        while iterations < scramble_iterations:
            tmp_scrambled = list(sequence)
            random.shuffle(left_sequence)
            random.shuffle(right_sequence)
            if left_sequence != left_original and right_sequence != right_original:
                tmp_scrambled[left_indices[0]:left_indices[1]+1] = left_sequence
                tmp_scrambled[right_indices[0]:right_indices[1]+1] = right_sequence
                scrambled_candidates.append(''.join(tmp_scrambled))
                iterations += 1

        best_recovery = 0
        for i in scrambled_candidates:
            tmp_structure, tmp_energy = linearfold(i)
            tmp_recovery = compute_bp_recovery(find_pairs(structure), find_pairs(tmp_structure))
            if  tmp_recovery > best_recovery:
                best_scrambled_recovery_sequence = i
                best_scrambled_recovery_structure = tmp_structure
                best_recovery = tmp_recovery
                print(tmp_recovery)
                print(linearfold(i)[0])
                alignments = pairwise2.align.globalxx(sequence, i) 
                print(format_alignment(*alignments[0])) 

        results['scrambled_rescue_sequence'] = best_scrambled_recovery_sequence
        results['scrambled_rescue_structure'] = best_scrambled_recovery_structure

    
    return results

In [34]:
generate_helix_flips(sequence, structure, helices[0], scramble_iterations=500)

Generating helix variants for
sequence: ACCCGACAGGAAACGGGGACUUAGGGGAACACGCUGGAGAUGCCAUGUGUGGCUGCCGAAGGUCACCAUCUCUCCUCAGUGGCUCCCCAGAGCAGGUGCUUUUAAGAACCCUGUUUCCUCUCAGA
structure:.......(((((((((((.....(((((.(((...(((((....(((.((((((......))))))))).)))))...)))..))))).((((....)))).......)))))))))))......
helix: [[7, 118], [8, 117], [9, 116], [10, 115], [11, 114], [12, 113], [13, 112], [14, 111], [15, 110], [16, 109], [17, 108]]
ACCCGACAGG---AAACGGGGA-CUUAGGGGAACACGCUGGAGAUGCCAUGUGUGGCUGCCGAAGGUCACCAUCUCUCCUCAGUGGCUCCCCAGAGCAGGUGCUUUUAAGAACCCUGUUUCCUCUCAGA
||||||| ||   |||   ||| ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
ACCCGAC-GGGGCAAA---GGAACUUAGGGGAACACGCUGGAGAUGCCAUGUGUGGCUGCCGAAGGUCACCAUCUCUCCUCAGUGGCUCCCCAGAGCAGGUGCUUUUAAGAACCCUGUUUCCUCUCAGA
  Score=121

ACCCGACAGGAAACGGGGACUUAGGGGAACACGCUGGAGAUGCCAUGUGUGGCUGCCGAAGGUCACCAUCUCUCCUCAGUGGCUCCCCAGAGCAGGUGCUUUUAAGAAC-CCUGUU-UCCUC-UCAGA
||||||||||||||||||||||||||||||||||||||||

{'flipped_rescue': 'ACCCGACGGGGCAAAGGAACUUAGGGGAACACGCUGGAGAUGCCAUGUGUGGCUGCCGAAGGUCACCAUCUCUCCUCAGUGGCUCCCCAGAGCAGGUGCUUUUAAGAAUCCUUUGCCCCCUCAGA',
 'flipped_rescue_structure': '.......(((((((((((.(((((((((.(((...(((((....(((.((((((......))))))))).)))))...)))..)))))(((((....)))))))))..)))))))))))......',
 'helix': [[7, 118],
  [8, 117],
  [9, 116],
  [10, 115],
  [11, 114],
  [12, 113],
  [13, 112],
  [14, 111],
  [15, 110],
  [16, 109],
  [17, 108]],
 'left_WT': 'AGGAAACGGGG',
 'left_flipped': 'ACCCGACGGGGCAAAGGAACUUAGGGGAACACGCUGGAGAUGCCAUGUGUGGCUGCCGAAGGUCACCAUCUCUCCUCAGUGGCUCCCCAGAGCAGGUGCUUUUAAGAACCCUGUUUCCUCUCAGA',
 'left_flipped_structure': '.(((....)))...((((((((.(((((.(((...(((((....(((.((((((......))))))))).)))))...)))..))))).)))((((.............)))).)))))......',
 'right_WT': 'CCCUGUUUCCU',
 'right_flipped': 'ACCCGACAGGAAACGGGGACUUAGGGGAACACGCUGGAGAUGCCAUGUGUGGCUGCCGAAGGUCACCAUCUCUCCUCAGUGGCUCCCCAGAGCAGGUGCUUUUAAGAAUCCUUUGUCCCCUCAGA',
 'right_flipped_structure': '...........

In [130]:
results = []
for helix in helices:
    print(helix)
    results.append(generate_helix_flips(sequence, structure, helix, scramble_iterations=250))

[[7, 118], [8, 117], [9, 116], [10, 115], [11, 114], [12, 113], [13, 112], [14, 111], [15, 110], [16, 109], [17, 108]]
Generating helix variants for
sequence: ACCCGACAGGAAACGGGGACUUAGGGGAACACGCUGGAGAUGCCAUGUGUGGCUGCCGAAGGUCACCAUCUCUCCUCAGUGGCUCCCCAGAGCAGGUGCUUUUAAGAACCCUGUUUCCUCUCAGA
structure:.......(((((((((((.....(((((.(((...(((((....(((.((((((......))))))))).)))))...)))..))))).((((....)))).......)))))))))))......
helix: [[7, 118], [8, 117], [9, 116], [10, 115], [11, 114], [12, 113], [13, 112], [14, 111], [15, 110], [16, 109], [17, 108]]
ACCCGACAGG---AAACGGGGA-CUUAGGGGAACACGCUGGAGAUGCCAUGUGUGGCUGCCGAAGGUCACCAUCUCUCCUCAGUGGCUCCCCAGAGCAGGUGCUUUUAAGAACCCUGUUUCCUCUCAGA
||||||| ||   |||   ||| ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
ACCCGAC-GGGGCAAA---GGAACUUAGGGGAACACGCUGGAGAUGCCAUGUGUGGCUGCCGAAGGUCACCAUCUCUCCUCAGUGGCUCCCCAGAGCAGGUGCUUUUAAGAACCCUGUUUCCUCUCAGA
  Score=121

ACCCGACAGGAAACGGGGACUUAGGGGAACACGCUGGAGAUGCCAUGUGU

In [131]:
columns = ['sequence', 'structure', 'helix',
                                           'left_WT', 'left_flipped', 'left_flipped_structure',
                                           'right_WT', 'right_flipped','right_flipped_structure',
                                           'flipped_rescue', 'flipped_rescue_structure',
                                           'scrambled_disruption_sequence', 'scrambled_disruption_structure',
                                           'scrambled_rescue_sequence', 'scrambled_rescue_structure']
results =   pd.DataFrame(results)
results.to_csv('test.csv')

In [132]:
all_indices = [[50,140],
               [155,275],
               [655,775],
               [800,840],
               [840,890],
               [1000,1030],
               [1105,1150],
               [1150,1260],
               [1280,1400],
               [1400,1445]
              ]

In [133]:
all_results = []
for indices in all_indices:
    sequence = master_sequence[indices[0]:indices[1]]
    structure = master_structure[indices[0]:indices[1]]
    pairs = find_pairs(structure)
    helices = find_helices(pairs)
    
    for helix in helices:
        print(helix)
        all_results.append(generate_helix_flips(sequence, structure, helix, scramble_iterations=100))

[[1, 86], [2, 85], [3, 84], [4, 83], [5, 82]]
[[20, 44], [21, 43], [22, 42]]
[[23, 40], [24, 39]]
[[26, 37], [27, 36]]
[[46, 75], [47, 74], [48, 73], [49, 72]]
[[51, 71], [52, 70], [53, 69], [54, 68]]
[[1, 86], [2, 85], [3, 84], [4, 83], [5, 82]]
Generating helix variants for
sequence: ACUGCCUUAAACGUUUAAUUCUACUUGCACCAAAUAGCUAGUUAGAGCAGACCCUCUCUUAAUCCCGUGGGGCUGUGAACGCGGCGGGGC
structure:.(((((..............(((((.((........)).)).))).((((.((((.............))))))))......)))))...
helix: [[1, 86], [2, 85], [3, 84], [4, 83], [5, 82]]
ACUGC--CUUAAACGUUUAAUUCUACUUGCACCAAAUAGCUAGUUAGAGCAGACCCUCUCUUAAUCCCGUGGGGCUGUGAACGCGGCGGGGC
||  |  |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
AC--CGUCUUAAACGUUUAAUUCUACUUGCACCAAAUAGCUAGUUAGAGCAGACCCUCUCUUAAUCCCGUGGGGCUGUGAACGCGGCGGGGC
  Score=88

ACUGCCUUAAACGUUUAAUUCUACUUGCACCAAAUAGCUAGUUAGAGCAGACCCUCUCUUAAUCCCGUGGGGCUGUGAACGCGGCGGGGC
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

In [134]:
all_results_df = pd.DataFrame(all_results)
all_results_df.to_csv('MBP3UTR_M2R_library_testing.csv', columns=columns)

In [135]:
linearfold('CCUUCGAUAGCUCAGCUGGUAGAGCGGAGGACUGUAGAUCCUUAGGUCGCUGGUUCGAUUCCGGCUCGAAGGACCA')

('(((((((..((((........)))).(((((.......))))).....(((((.......))))))))))))....',
 6.36)

In [144]:
linearfold('CUUCGAUAGCUCAGCUGGUAGAGCGGAGGACUGUAGAUCCUUAGGUCGCUGGUUCGAUUCCGGCUCGAAGGACCA')

('((((((...(((.(((.....))).)))((((..((...))..))))(((((.......))))))))))).....',
 -24.4)